In [70]:
# first we do is for hr bills, congress nr 112
import json

lo_bills = []

# This is our module to get all the bill file paths.
from bill_paths import get_bill_file_paths 

# All the bills are organized alphabetically so instead of hr1, hr2, hr3 you'll get hr1, hr10, hr100 but the order is not relevant to us.
bill_paths = get_bill_file_paths(112)

for path in bill_paths:
    with open(path) as json_file:
        bill = json.load(json_file)
    # First check if the bill contains all the items we want for our project.
        if ("bill_id" and "official_title" and "subject_top_term" and "summary") in bill.keys():
            dictionary = {}
            dictionary["bill_id"] = bill["bill_id"]
    # We use the official title 
            dictionary["title"] = bill["official_title"] 
    # Most bills have many subjects so we decid to only look at the top term for clarity
            dictionary["subject"] = bill["subjects_top_term"]
            summary_ = bill["summary"]
            dictionary["summary"] = summary_["text"]
    
    # Appended to the list of bills 
        lo_bills.append(dictionary)
            
#per bill we create a dictionary containing the following data:
#Bill id, Title, Summary
# then, we create a list of all these dictionaries

TypeError: 'NoneType' object is not subscriptable

In [67]:
import csv

with open('112.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter='|')
    filewriter.writerow(['bill_id', 'title', 'subject','summary'])
    for bill in lo_bills:
        filewriter.writerow([bill['bill_id'], bill['title'], bill['subject'], bill['summary']])

KeyError: 'summary'